In [1]:
import pandas as pd
import numpy as np
from src.utils.elo import Elo

## WIP - I want to calculate ELO ratings and add these as a a features into gameweek data

In [29]:
gameweek_df = pd.read_csv('data/processed/combined_gameweeks.csv')

/opt/homebrew/Caskroom/miniconda/base/envs/ravelin36/lib/python3.6/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (25,57,58) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [30]:
len(gameweek_df.drop_duplicates(subset=['team', 'opponent_team','gw', 'season']))

2312

In [31]:
len(gameweek_df.drop_duplicates(subset=['team','gw', 'season']))

1023

In [38]:
gameweek_dedup = gameweek_df.groupby(['team','gw','season','opponent_team'],as_index=False).agg({'name':'count',
                                                            'team_h_score':'first',
                                                            'team_a_score':'first', 
                                                            'was_home':'first'}).sort_values(by=['opponent_team',
                                                                                                 'season',
                                                                                                 'gw',
                                                                                                 'name']).drop_duplicates(subset=['opponent_team','season','gw'],keep='last')

In [43]:
len(gameweek_dedup)

956

In [10]:
gameweek_df.groupby(['gw','season','team'], as_index=False)['name'].count().min()

gw              1
season    2020-21
team      Arsenal
name           23
dtype: object

In [ ]:
teams = pd.read_csv(f'data/Fantasy-Premier-League/data/master_team_list.csv').rename(columns={'team':'opponent_team','team_name':'opponent_name'})

with_opponents = gameweek_df.merge(teams,on=['season','opponent_team'],how='left')

fixtures = with_opponents[['team','opponent_name','season','gw','team_h_score','team_a_score', 'was_home']].loc[with_opponents.team.notna()].drop_duplicates().copy()

fixtures['opp_team'] = 'opp_' + fixtures.opponent_name # so we can merge and update

fixtures['home_outcome'] =(np.clip(fixtures['team_h_score'] -fixtures['team_a_score'],-1,1) + 1) / 2

fixtures['outcome'] = np.where(fixtures.was_home, fixtures.home_outcome, 1-fixtures.home_outcome) # there must be a simpler way

elo =Elo(fixtures,k_factor=20, team_h='team',team_a='opp_team')
historic_elo, elo_ratings =elo.process_all_fixtures()

historic_elo[['team', 'season', 'gw','elo_e']]

In [ ]:
pd.Series(elo_ratings).sort_values(ascending=False)